# script to update group quarters data for soundcast input

returns group quarters data aggregated to the TAZ level

In [ ]:
import pandas as pd
import os, sys, time
import h5py
from sqlalchemy import create_engine, text
import logging


year = '2023'
# group quarters rate from emme_configuration.toml
group_quarters_rate = 0.0034

conn = create_engine('sqlite:///R:/e2projects_two/SoundCast/Inputs/dev/db/soundcast_inputs_2023.db')

In [2]:
# read data
new_group_quarters_data = "R:/e2projects_two/2023_base_year/group_quarters/gqpop2023_bySoundCastType.csv"
total_gq = pd.read_csv(new_group_quarters_data)

total_gq.columns

Index(['Geoid20', 'TAZ', 'group_quarters', 'dorm_share', 'military_share',
       'other_share', 'year'],
      dtype='object')

In [5]:
total_gq_df = total_gq.copy()

total_gq_df[["dorm_share", "military_share", "other_share"]] = total_gq_df[
    ["dorm_share", "military_share", "other_share"]
].astype("float")

# Calculate the Inputs for the Year of the model
max_input_year = total_gq_df["year"].max()

if int(year) <= max_input_year:
    total_gq_df = total_gq_df[
        total_gq_df["year"] == int(year)
    ]

else:
    # Factor group quarters at an annual rate
    total_gq_df = total_gq_df[total_gq_df["year"] == int(max_input_year)]
    total_gq_df["group_quarters"] = total_gq_df["group_quarters"] * (
        1
        + (
            group_quarters_rate
            * (int(year) - max_input_year)
        )
    )

total_gq_df = total_gq_df[
    ["TAZ", "dorm_share", "military_share", "other_share", "group_quarters"]
]

total_gq_df["dorms"] = total_gq_df["group_quarters"] * total_gq_df["dorm_share"]
total_gq_df["military"] = (
    total_gq_df["group_quarters"] * total_gq_df["military_share"]
)
total_gq_df["other"] = total_gq_df["group_quarters"] * total_gq_df["other_share"]

# Merge with the Block/Taz dataframe and trim down the columns
total_gq_df = total_gq_df[["TAZ", "dorms", "military", "other"]].rename(
    columns={"TAZ": "taz"} # rename taz to match the database column name
)
total_gq_df['year'] = "2023"
total_gq_df.to_csv('R:/e2projects_two/SoundCast/Inputs/db_inputs/group_quarters.csv', index=False)

In [6]:
total_gq_df.columns

Index(['taz', 'dorms', 'military', 'other', 'year'], dtype='object')